In [12]:
# this is a simple script to load the OpenAI API key from a .env file
# make sure to create a .env file with your OpenAI API key
# also .env file is added to .gitignore so it won't be pushed to the git repository
from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()
# Get the OpenAI API key
openai_api_key = os.getenv('OPENAI_API_KEY')
print("OpenAI API Key loaded successfully.")
# Get the stock API key
stock_api_key = os.getenv('ALPHA_VANTAGE_API_KEY')
print("Stock API Key loaded successfully.")

OpenAI API Key loaded successfully.
Stock API Key loaded successfully.


In [14]:
from langchain_openai import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
import math
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
import requests

# define a calculator function and tool
def calculator(expression: str):
    """Evaluates a math expression and returns the result."""
    return eval(expression)

calculator_tool = Tool(
    name="Calculator",
    func=calculator,
    description="A simple calculator that can evaluate math expressions."
)

# Create a Wikipedia search tool
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Create a stock query *custom* tool
def get_stock_price(symbol: str):
    """Fetches the latest stock price for a given ticker symbol."""
    api_key = stock_api_key  # Replace with your API key
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={api_key}"    
    response = requests.get(url)
    data = response.json()    
    if "Global Quote" in data:
        price = data["Global Quote"].get("05. price", "N/A")
        return f"The current price of {symbol.upper()} is ${price}."
    else:
        return "Invalid stock symbol or API limit reached."

# Define the stock tool
stock_tool = Tool(
    name="Stock Price Checker",
    func=get_stock_price,
    description="Fetches the latest stock price for a given ticker symbol (e.g., AAPL, TSLA, MSFT)."
)




# Define the LLM model
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Extend the agent with both tools
agent = initialize_agent(
    tools=[calculator_tool, wikipedia, stock_tool],  
    llm=llm,  
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  
    verbose=True  
)

## use the tools
# Ask a question that requires Wikipedia search
#response = agent.invoke("Who was Albert Einstein?")
#print(response)

#response = agent.invoke("What is 34*33")
#print(response)

#response = agent.invoke("What is stock price for LULU ?")
#print(response)
# Generate a set of questions for the agent
questions = {
    "math_question": "What is 45*67?",
    "wiki_question": "Who was Nikola Tesla?",
    "stock_question": "What is the stock price for AAPL?"
}

# Use the agent to answer the questions
for key, question in questions.items():
    response = agent.invoke(question)
    print(f"{key}: {response}")



> Entering new AgentExecutor chain...
This is a simple multiplication problem. I can use the Calculator tool to solve it.
Action: Calculator
Action Input: "45*67"
Observation: 3015
Thought:I now know the final answer.
Final Answer: The result of 45*67 is 3015.

> Finished chain.
math_question: {'input': 'What is 45*67?', 'output': 'The result of 45*67 is 3015.'}


> Entering new AgentExecutor chain...
This question is asking for information about a historical figure, Nikola Tesla. I can use the wikipedia tool to find this information.
Action: wikipedia
Action Input: Nikola Tesla
Observation: Page: Nikola Tesla
Summary: Nikola Tesla (; Serbian Cyrillic: Никола Тесла [nǐkola têsla]; 10 July 1856 – 7 January 1943) was a Serbian-American engineer, futurist, and inventor. He is known for his contributions to the design of the modern alternating current (AC) electricity supply system.
Born and raised in the Austrian Empire, Tesla first studied engineering and physics in the 1870s without r